In [3]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from config import gkey

### Store Part I results into DataFrame
Load the csv exported in Part I to a DataFrame


In [4]:
Cities_DF = pd.read_csv('Cities.csv')
Cities_DF.head()

,City_ID,City,Country,Temperature,Humidity,Cloudiness,Windspeed,Latitude,Longitude,Date
0,0,tuktoyaktuk,CA,-24.50,100,91,5.01,69.4541,-133.0374,1647859737
1,1,qaanaaq,GL,-6.30,93,100,10.83,77.4840,-69.3632,1647859737
2,2,albany,US,36.68,80,46,1.01,42.6001,-73.9662,1647859572
3,3,nouadhibou,MR,66.18,59,0,24.16,20.9310,-17.0347,1647859739
4,4,port elizabeth,ZA,87.10,59,20,6.91,-33.9180,25.5701,1647859740


### Humidity Heatmap
Configure gmaps.
Use the Lat and Lng as locations and Humidity as the weight.
Add Heatmap layer to map.

In [6]:
#configure gmaps
gmaps.configure(api_key=gkey)

#get locations and humidity
city_locations = Cities_DF[['Latitude', 'Longitude']].astype(float)
humidity = Cities_DF['Humidity'].astype(float)

#format layout of map
layout = {
    'width': '1000px',
    'height': '600px',
    'border': '2px solid black',
    'padding': '2px'}

#add heatmap
fig = gmaps.figure(map_type='HYBRID', layout=layout)

heat_layer = gmaps.heatmap_layer(city_locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(border='2px solid black', height='600px', padding='2px', width='1000px'))

### Create new DataFrame fitting weather criteria
Narrow down the cities to fit weather conditions.
Drop any rows will null values.

In [12]:
myFavoriteWeather = Cities_DF.loc[(Cities_DF['Temperature'] >= 50) & (Cities_DF['Temperature'] <= 80) & 
                                  (Cities_DF['Windspeed'] <= 10) & (Cities_DF['Humidity'] <= 40)]
myFavoriteWeather

,City_ID,City,Country,Temperature,Humidity,Cloudiness,Windspeed,Latitude,Longitude,Date
18,18,kumluca,TR,52.05,40,20,7.70,36.3703,30.2869,1647859751
76,76,bayir,TR,50.52,34,2,5.48,37.2687,28.2167,1647859800
119,119,buraydah,SA,77.45,20,0,6.82,26.3260,43.9750,1647859837
159,159,deggendorf,DE,58.78,31,0,5.84,48.8333,12.9667,1647859870
210,210,garmsar,IR,70.63,15,91,4.16,35.2182,52.3409,1647859912
244,244,mabopane,ZA,70.18,33,79,3.94,-25.4977,28.1007,1647859941
312,312,de aar,ZA,77.34,33,1,2.86,-30.6497,24.0123,1647859999
316,316,chaman,PK,74.80,11,0,5.06,30.9236,66.4512,1647860002
435,435,tomatlan,MX,61.63,34,24,2.64,19.9333,-105.2500,1647860105
556,556,sabzevar,IR,67.28,21,0,0.00,36.2126,57.6819,1647860207


### Hotel Map
Store into variable named hotel_df
Add a "Hotel Name" column to the DataFrame.
Set parameters to search for hotels with 5000 meters.
Hit the Google Places API for each city's coordinates.
Store the first Hotel result into the DataFrame.
Plot markers on top of the heatmap

In [13]:
hotel_df = myFavoriteWeather
hotel_df.reset_index(drop=True, inplace=True)
hotel_df

,City_ID,City,Country,Temperature,Humidity,Cloudiness,Windspeed,Latitude,Longitude,Date
0,18,kumluca,TR,52.05,40,20,7.70,36.3703,30.2869,1647859751
1,76,bayir,TR,50.52,34,2,5.48,37.2687,28.2167,1647859800
2,119,buraydah,SA,77.45,20,0,6.82,26.3260,43.9750,1647859837
3,159,deggendorf,DE,58.78,31,0,5.84,48.8333,12.9667,1647859870
4,210,garmsar,IR,70.63,15,91,4.16,35.2182,52.3409,1647859912
5,244,mabopane,ZA,70.18,33,79,3.94,-25.4977,28.1007,1647859941
6,312,de aar,ZA,77.34,33,1,2.86,-30.6497,24.0123,1647859999
7,316,chaman,PK,74.80,11,0,5.06,30.9236,66.4512,1647860002
8,435,tomatlan,MX,61.63,34,24,2.64,19.9333,-105.2500,1647860105
9,556,sabzevar,IR,67.28,21,0,0.00,36.2126,57.6819,1647860207


In [15]:
hotel_df['Hotel Name'] = ''

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    "radius": 50000,
    "rankby": "prominence",
    "types": "lodging",
    "keyword": "hotel",
    "key": gkey,
}

# use iterrows to iterate through pandas dataframe
for index, row in hotel_df.iterrows():

    # get variables type from df
    lat = row['Latitude']
    lng = row['Longitude']
    city = row['City']
    params["location"] = f'{lat},{lng}'
        
    # assemble url and make API request
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    try:
        print(f"Closest {city} hotel is {results[0]['name']}.")
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        
    except:
        print(f"Missing field/result for {city}... skipping.")
        
    print("------------")

C:\Users\2mini\AppData\Local\Temp/ipykernel_20736/2985693038.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df['Hotel Name'] = ''


Closest kumluca hotel is Beldibi Hotel.
------------


C:\Users\2mini\anaconda3\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


Closest bayir hotel is BLUE BAY PLATINUM HOTEL.
------------
Closest buraydah hotel is Ramada by Wyndham Al Qassim.
------------
Closest deggendorf hotel is Hotel Gasthof Höttl Deggendorf.
------------
Closest garmsar hotel is Ssdg hotel.
------------
Closest mabopane hotel is Sheraton Pretoria Hotel.
------------
Closest de aar hotel is Hanover Lodge Hotel.
------------
Closest chaman hotel is Al Hikmat Hotel.
------------
Closest tomatlan hotel is Las Alamandas.
------------
Closest sabzevar hotel is Sabzevar Tourism hotel.
------------


In [16]:
hotel_df = hotel_df[hotel_df['Hotel Name'].notna()]
hotel_df = hotel_df.rename(columns={"Latitude": "Lat", "Longitude": "Lng"})
hotel_df

,City_ID,City,Country,Temperature,Humidity,Cloudiness,Windspeed,Lat,Lng,Date,Hotel Name
0,18,kumluca,TR,52.05,40,20,7.70,36.3703,30.2869,1647859751,Beldibi Hotel
1,76,bayir,TR,50.52,34,2,5.48,37.2687,28.2167,1647859800,BLUE BAY PLATINUM HOTEL
2,119,buraydah,SA,77.45,20,0,6.82,26.3260,43.9750,1647859837,Ramada by Wyndham Al Qassim
3,159,deggendorf,DE,58.78,31,0,5.84,48.8333,12.9667,1647859870,Hotel Gasthof Höttl Deggendorf
4,210,garmsar,IR,70.63,15,91,4.16,35.2182,52.3409,1647859912,Ssdg hotel
5,244,mabopane,ZA,70.18,33,79,3.94,-25.4977,28.1007,1647859941,Sheraton Pretoria Hotel
6,312,de aar,ZA,77.34,33,1,2.86,-30.6497,24.0123,1647859999,Hanover Lodge Hotel
7,316,chaman,PK,74.80,11,0,5.06,30.9236,66.4512,1647860002,Al Hikmat Hotel
8,435,tomatlan,MX,61.63,34,24,2.64,19.9333,-105.2500,1647860105,Las Alamandas
9,556,sabzevar,IR,67.28,21,0,0.00,36.2126,57.6819,1647860207,Sabzevar Tourism hotel


In [17]:
# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [18]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(border='2px solid black', height='600px', padding='2px', width='1000px'))